## Шаг 1. Создание таблиц с перечисленными структурами

Данные остались те же, что и ранее использовались в `HW_2`

![Итоговый вид таблиц](tables.png)

![Примеры загруженных данных](data_tables_examples.png)

## Шаг 2. Подготовка окружения и создание подключения

In [1]:
%pip install "psycopg[binary]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg

from psycopg.rows import TupleRow

In [3]:
connection = psycopg.connect(
    dbname="hw_2",
    user="postgres",
    password="1234qaz",
    host="localhost",
    port="5432",
)

cur = connection.cursor()

def resalt_output(records: list[TupleRow]) -> None:
    """Вывод информации о записях БД."""
    if not records:
        print("Запрос не дал результатов.")
    else:
        for i, row in enumerate(records):
            print(f"Вывод строки #{i+1}: {row}")

## Шаг 3. Выполнение запросов

1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [4]:
query_1 = """
    select
        c.job_industry_category,
        count(c.customer_id) as customers_count
    from customer c 
    where c.job_industry_category is not null
    group by c.job_industry_category
    order by customers_count desc;
"""

cur.execute(query_1)

resalt_output(cur.fetchall())

Вывод строки #1: ('Manufacturing', 799)
Вывод строки #2: ('Financial Services', 774)
Вывод строки #3: ('n/a', 656)
Вывод строки #4: ('Health', 602)
Вывод строки #5: ('Retail', 358)
Вывод строки #6: ('Property', 267)
Вывод строки #7: ('IT', 223)
Вывод строки #8: ('Entertainment', 136)
Вывод строки #9: ('Argiculture', 113)
Вывод строки #10: ('Telecommunications', 72)


2. Найти общую сумму дохода (list_price*quantity) по всем подтвержденным заказам за каждый месяц по сферам деятельности клиентов. Отсортировать результат по году, месяцу и сфере деятельности.

In [5]:
query_2 = """
    select
        date_trunc('year', o.order_date) as year_order,
        date_trunc('month', o.order_date) as month_order, 
        c.job_industry_category,
        sum(oi.item_list_price_at_sale * oi.quantity) as total
    from customer c 
    join orders o on o.customer_id = c.customer_id
    join order_items oi on oi.order_id = o.order_id 
    where o.order_status = 'Approved'
    group by
        year_order,
        month_order,
        c.job_industry_category
    order by
        year_order,
        month_order,
        c.job_industry_category;
"""

cur.execute(query_2)

resalt_output(cur.fetchall())

Вывод строки #1: (datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Argiculture', 232148.25)
Вывод строки #2: (datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Entertainment', 342541.22)
Вывод строки #3: (datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Financial Services', 2032708.8)
Вывод строки #4: (datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Health', 1570012.4)
Вывод строки #5: (datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'IT', 604949.44)
Вывод строки #6: (datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2017, 1, 1, 

3. Вывести количество уникальных онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. Включить бренды, у которых нет онлайн-заказов от IT-клиентов, — для них должно быть указано количество 0.

In [6]:
query_3 = """
    select
        p.brand,
        count(distinct 
            case
                when o.order_status = 'Approved' 
                    and o.online_order is true 
                    and c.job_industry_category = 'IT'
                then o.order_id
            end
        ) as unique_online_orders
    from product p
    left join order_items oi on p.product_id = oi.product_id
    left join orders o on oi.order_id = o.order_id
    left join customer c on o.customer_id = c.customer_id
    group by p.brand
    order by p.brand;
"""

cur.execute(query_3)

resalt_output(cur.fetchall())

Вывод строки #1: ('', 44)
Вывод строки #2: ('Giant Bicycles', 194)
Вывод строки #3: ('Norco Bicycles', 173)
Вывод строки #4: ('OHM Cycles', 174)
Вывод строки #5: ('Solex', 202)
Вывод строки #6: ('Trek Bicycles', 160)
Вывод строки #7: ('WeareA2B', 171)


4. Найти по всем клиентам: сумму всех заказов (общего дохода), максимум, минимум и количество заказов, а также среднюю сумму заказа по каждому клиенту. Отсортировать результат по убыванию суммы всех заказов и количества заказов. Выполнить двумя способами: используя только GROUP BY и используя только оконные функции. Сравнить результат.

In [7]:
# используя только GROUP BY
query_4_1 = """
    with order_totals as (
        select 
            o.customer_id,
            o.order_id,
            sum(oi.quantity * oi.item_list_price_at_sale) as total
        from orders o 
        join order_items oi on o.order_id = oi.order_id 
        group by o.customer_id, o.order_id
    )
    select
        distinct ot.customer_id,
        sum(ot.total) as total_income,
        max(ot.total) as max_order_price,
        min (ot.total) as min_order_price,
        count(ot.order_id) as orders_count,
        avg(ot.total) as avg_order_price
    from order_totals ot
    group by ot.customer_id
    order by total_income desc, orders_count desc;
"""

cur.execute(query_4_1)

resalt_output(cur.fetchall())

Вывод строки #1: (2183, 136632.45, 20056.6, 1073.07, 14, 9759.461478097099)
Вывод строки #2: (1597, 133657.06, 20914.7, 1720.7, 12, 11138.088195800781)
Вывод строки #3: (941, 129789.92, 20914.7, 2115.02, 10, 12978.993896484375)
Вывод строки #4: (1129, 129189.484, 19773.6, 1743.72, 13, 9937.65243765024)
Вывод строки #5: (637, 109334.734, 17796.24, 360.4, 13, 8410.364624023438)
Вывод строки #6: (2309, 107476.68, 16353.0, 991.44, 12, 8956.390040079752)
Вывод строки #7: (3015, 106182.33, 20914.7, 3111.16, 10, 10618.232983398437)
Вывод строки #8: (1329, 101439.08, 17934.3, 543.39, 11, 9221.733575994318)
Вывод строки #9: (2046, 100891.35, 18823.23, 2667.55, 9, 11210.15025499132)
Вывод строки #10: (2615, 99880.695, 19773.6, 1149.28, 9, 11097.8544921875)
Вывод строки #11: (2914, 98618.766, 19929.3, 471.26, 12, 8218.230875651041)
Вывод строки #12: (151, 96678.19, 17936.371, 6498.76, 8, 12084.772521972656)
Вывод строки #13: (2637, 94577.195, 20914.7, 500.43, 11, 8597.9262639826)
Вывод строки #14

In [8]:
# используя только оконные функции
query_4_2 = """
    with items_total as (
        select 
            o.customer_id,
            o.order_id,
            oi.quantity * oi.item_list_price_at_sale as total
        from orders o 
        join order_items oi on o.order_id = oi.order_id 
    ),
    orders_total as (
        select 
            it.customer_id,
            it.order_id,
            sum(it.total) over (partition by it.order_id) as order_total
        from items_total it
    )
    select 
        distinct ot.customer_id,
        sum(ot.order_total) over (partition by ot.customer_id) as total_income,
        max(ot.order_total) over (partition by ot.customer_id) as max_order_price,
        min(ot.order_total) over (partition by ot.customer_id) as min_order_price,
        count(ot.order_id) over (partition by ot.customer_id) as orders_count,
        avg(ot.order_total) over (partition by ot.customer_id) as avg_order_price
    from orders_total ot
    order by total_income desc, orders_count desc;
"""

cur.execute(query_4_2)

resalt_output(cur.fetchall())

Вывод строки #1: (2183, 136632.47, 20056.6, 1073.07, 14, 9759.461478097099)
Вывод строки #2: (1597, 133657.05, 20914.7, 1720.7, 12, 11138.088195800781)
Вывод строки #3: (941, 129789.95, 20914.7, 2115.02, 10, 12978.993896484375)
Вывод строки #4: (1129, 129189.47, 19773.6, 1743.72, 13, 9937.65243765024)
Вывод строки #5: (637, 109334.734, 17796.24, 360.4, 13, 8410.364624023438)
Вывод строки #6: (2309, 107476.68, 16353.0, 991.44, 12, 8956.390040079752)
Вывод строки #7: (3015, 106182.33, 20914.7, 3111.16, 10, 10618.232983398437)
Вывод строки #8: (1329, 101439.08, 17934.3, 543.39, 11, 9221.733575994318)
Вывод строки #9: (2046, 100891.34, 18823.23, 2667.55, 9, 11210.15025499132)
Вывод строки #10: (2615, 99880.69, 19773.6, 1149.28, 9, 11097.8544921875)
Вывод строки #11: (2914, 98618.77, 19929.3, 471.26, 12, 8218.230875651041)
Вывод строки #12: (151, 96678.18, 17936.371, 6498.76, 8, 12084.772521972656)
Вывод строки #13: (2637, 94577.19, 20914.7, 500.43, 11, 8597.9262639826)
Вывод строки #14: (5

5. Найти имена и фамилии клиентов с топ-3 минимальной и топ-3 максимальной суммой транзакций за весь период (учесть клиентов, у которых нет заказов, приняв их сумму транзакций за 0).

In [9]:
query_5 = """
    with customers_total as (
        select 
            c.customer_id,
            c.first_name,
            c.last_name,
            coalesce(sum(oi.quantity * oi.item_list_price_at_sale), 0) as total
        from customer c 
        left join orders o on c.customer_id = o.customer_id
        left join order_items oi on o.order_id = oi.order_id 
        group by 
            c.customer_id,
            c.first_name,
            c.last_name
    )
    select * from (
        select *
        from customers_total
        order by total desc
        limit 3
    )
    union all
    select * from (
        select *
        from customers_total
        where total > 0
        order by total asc
        limit 3
    )
    order by total;
"""

cur.execute(query_5)

resalt_output(cur.fetchall())

Вывод строки #1: (3292, 'Hamlen', 'Slograve', 60.34)
Вывод строки #2: (2532, 'Milli', 'Hubbert', 71.49)
Вывод строки #3: (301, 'Hildy', 'Bilbrook', 432.94)
Вывод строки #4: (941, 'Tye', 'Doohan', 129789.945)
Вывод строки #5: (1597, 'Jeffry', 'Slowly', 133657.06)
Вывод строки #6: (2183, 'Jillie', 'Fyndon', 136632.45)


6. Вывести только вторые транзакции клиентов (если они есть) с помощью оконных функций. Если у клиента меньше двух транзакций, он не должен попасть в результат.

In [10]:
query_6 = """
    with numbered_orders as (
        select
            *,
            row_number() over (partition by customer_id order by order_date) as order_num
        from orders
    )
    select *
    from numbered_orders
    where order_num = 2
    order by order_id;
"""

cur.execute(query_6)

resalt_output(cur.fetchall())

Вывод строки #1: (7, 1542, datetime.date(2017, 4, 21), True, 'Approved', 2)
Вывод строки #2: (11, 1986, datetime.date(2017, 1, 17), False, 'Approved', 2)
Вывод строки #3: (17, 2426, datetime.date(2017, 4, 3), False, 'Approved', 2)
Вывод строки #4: (20, 3002, datetime.date(2017, 1, 28), True, 'Approved', 2)
Вывод строки #5: (23, 2001, datetime.date(2017, 4, 8), True, 'Approved', 2)
Вывод строки #6: (43, 488, datetime.date(2017, 3, 17), False, 'Cancelled', 2)
Вывод строки #7: (49, 1251, datetime.date(2017, 6, 17), False, 'Approved', 2)
Вывод строки #8: (51, 3108, datetime.date(2017, 3, 24), False, 'Approved', 2)
Вывод строки #9: (55, 3140, datetime.date(2017, 9, 18), False, 'Approved', 2)
Вывод строки #10: (61, 1839, datetime.date(2017, 2, 24), False, 'Approved', 2)
Вывод строки #11: (68, 1498, datetime.date(2017, 9, 30), True, 'Approved', 2)
Вывод строки #12: (70, 2192, datetime.date(2017, 5, 6), True, 'Approved', 2)
Вывод строки #13: (73, 2430, datetime.date(2017, 7, 23), False, 'Appro

7. Вывести имена, фамилии и профессии клиентов, а также длительность максимального интервала (в днях) между двумя последовательными заказами. Исключить клиентов, у которых только один или меньше заказов.

In [11]:
query_7 = """
    with orders_with_prev_date as (
        select
            o.customer_id,
            c.first_name,
            c.last_name,
            c.job_title,
            o.order_date,
            lag(o.order_date) over (partition by o.customer_id order by o.order_date) as prev_order_date
        from orders o
        join customer c on o.customer_id = c.customer_id
    ),
    order_intervals as (
        select
            customer_id,
            first_name,
            last_name,
            job_title,
            order_date - prev_order_date as days_between_orders
        from orders_with_prev_date
        where prev_order_date is not null
    ),
    max_intervals as (
        select
            customer_id,
            first_name,
            last_name,
            job_title,
            max(days_between_orders) as max_interval_days
        from order_intervals
        group by 
            customer_id,
            first_name,
            last_name,
            job_title
    )
    select
        first_name,
        last_name,
        job_title,
        max_interval_days
    from max_intervals
    order by max_interval_days desc;
"""

cur.execute(query_7)

resalt_output(cur.fetchall())

Вывод строки #1: ('Susanetta', '', 'Legal Assistant', 357)
Вывод строки #2: ('Royall', 'Terris', 'Geological Engineer', 330)
Вывод строки #3: ('Stoddard', 'Giacomoni', 'Structural Analysis Engineer', 330)
Вывод строки #4: ('Gregorius', 'Cockram', 'Data Coordiator', 330)
Вывод строки #5: ('Bearnard', 'Letixier', '', 329)
Вывод строки #6: ('Caralie', 'Sellors', 'Senior Editor', 321)
Вывод строки #7: ('Debee', 'Martynov', 'Senior Editor', 320)
Вывод строки #8: ('Genni', 'Larway', 'Environmental Specialist', 314)
Вывод строки #9: ('Timmie', 'Lenden', '', 310)
Вывод строки #10: ('Carolynn', 'Samsin', 'Pharmacist', 310)
Вывод строки #11: ('Franz', 'Craddy', '', 310)
Вывод строки #12: ('Jodee', 'Judkins', 'Recruiting Manager', 306)
Вывод строки #13: ('Ashia', 'Muzzi', 'Mechanical Systems Engineer', 306)
Вывод строки #14: ('Heywood', 'Sollett', 'Tax Accountant', 305)
Вывод строки #15: ('Cleveland', 'Islep', 'Software Engineer II', 299)
Вывод строки #16: ('Sheilah', 'Blackmore', '', 297)
Вывод 

8. Найти топ-5 клиентов (по общему доходу) в каждом сегменте благосостояния (wealth_segment). Вывести имя, фамилию, сегмент и общий доход. Если в сегменте менее 5 клиентов, вывести всех.

In [12]:
query_8 = """
    with customers_total as (
        select 
            c.customer_id,
            c.first_name,
            c.last_name,
            c.wealth_segment,
            coalesce(sum(oi.quantity * oi.item_list_price_at_sale), 0) as orders_total
        from customer c 
        left join orders o on c.customer_id = o.customer_id
        left join order_items oi on o.order_id = oi.order_id 
        group by 
            c.customer_id,
            c.first_name,
            c.last_name,
            c.wealth_segment
    ),
    customers_range_by_segments as (
        select
            *,
            row_number() over (partition by wealth_segment order by orders_total desc) as range_by_segment
        from customers_total
    )
    select
        first_name,
        last_name,
        wealth_segment,
        orders_total
    from customers_range_by_segments
    where range_by_segment <= 5
    order by wealth_segment, orders_total desc;
"""

cur.execute(query_8)

resalt_output(cur.fetchall())

Вывод строки #1: ('Jeffry', 'Slowly', 'Affluent Customer', 133657.06)
Вывод строки #2: ('Tye', 'Doohan', 'Affluent Customer', 129789.945)
Вывод строки #3: ('Herc', 'McIlhone', 'Affluent Customer', 107476.69)
Вывод строки #4: ('Queenie', 'Flips', 'Affluent Customer', 106182.33)
Вывод строки #5: ('Jessamine', 'Brazear', 'Affluent Customer', 98618.77)
Вывод строки #6: ('Mercy', 'Wilsone', 'High Net Worth', 109334.74)
Вывод строки #7: ('Lockwood', 'Exroll', 'High Net Worth', 92405.18)
Вывод строки #8: ('Linell', '', 'High Net Worth', 91450.18)
Вывод строки #9: ('Gayelord', 'Lipman', 'High Net Worth', 90493.06)
Вывод строки #10: ('Jonell', 'Gon', 'High Net Worth', 87555.7)
Вывод строки #11: ('Jillie', 'Fyndon', 'Mass Customer', 136632.45)
Вывод строки #12: ('Hercule', '', 'Mass Customer', 129189.49)
Вывод строки #13: ('Wendy', 'Randlesome', 'Mass Customer', 101439.06)
Вывод строки #14: ('Charis', 'Maas', 'Mass Customer', 100891.35)
Вывод строки #15: ('Cordelia', '', 'Mass Customer', 99880.6

In [13]:
connection.close()